In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np


Using TensorFlow backend.


In [3]:
class GAN():
    
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        
        optimizer = Adam(0.0002, 0.5)
        
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
        
        self.generator = self.build_generator()
        
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        
        self.discriminator.trainable = False
        
        validity = self.discriminator(img)
        
        self.combined = Model(z, validity)
        
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
    def build_generator(self):
        
        model = Sequential()
        
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))
        
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise, img)
    
    
    def build_discriminator(self):
        
        model = Sequential()
        
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        
        img = Input(shape=self.img_shape)
        validity = model(img)
        
        return Model(img, validity)
    
    def train(self, epochs, batch_size=128, sample_interval=50):
        
        (X_train, _), (_, _) = mnist.load_data()
        
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)
        
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):
            
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            gen_imgs = self.generator.predict(noise)
            
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            g_loss = self.combined.train_on_batch(noise, valid)
            
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
            
    def sample_images(self, epoch):

        r, c = 5, 5

        noise = np.random.normal(0, 1, (r * c, self.latent_dim))

        gen_imgs = self.generator.predict(noise)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0

        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(".\\GAN_mnist\\%d.png" % epoch)
        plt.close()
            
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)
        
        
        

        
        

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 

c:\users\wojtek\appdata\local\conda\conda\envs\deep learning\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.623339, acc.: 54.69%] [G loss: 0.572458]
1 [D loss: 0.481532, acc.: 59.38%] [G loss: 0.688583]
2 [D loss: 0.378092, acc.: 76.56%] [G loss: 0.816634]
3 [D loss: 0.269403, acc.: 92.19%] [G loss: 0.975424]
4 [D loss: 0.228299, acc.: 93.75%] [G loss: 1.115721]
5 [D loss: 0.220162, acc.: 95.31%] [G loss: 1.305753]
6 [D loss: 0.159195, acc.: 100.00%] [G loss: 1.436540]
7 [D loss: 0.154170, acc.: 100.00%] [G loss: 1.623794]
8 [D loss: 0.153380, acc.: 100.00%] [G loss: 1.713285]
9 [D loss: 0.121269, acc.: 100.00%] [G loss: 1.847941]
10 [D loss: 0.102056, acc.: 100.00%] [G loss: 1.931719]
11 [D loss: 0.099410, acc.: 100.00%] [G loss: 1.958923]
12 [D loss: 0.097080, acc.: 100.00%] [G loss: 2.119991]
13 [D loss: 0.079169, acc.: 100.00%] [G loss: 2.164098]
14 [D loss: 0.066249, acc.: 100.00%] [G loss: 2.261892]
15 [D loss: 0.070214, acc.: 100.00%] [G loss: 2.220029]
16 [D loss: 0.065227, acc.: 100.00%] [G loss: 2.362289]
17 [D loss: 0.063458, acc.: 100.00%] [G loss: 2.483837]
18 [D lo

146 [D loss: 0.186525, acc.: 93.75%] [G loss: 4.914987]
147 [D loss: 1.709131, acc.: 31.25%] [G loss: 2.460320]
148 [D loss: 1.726514, acc.: 60.94%] [G loss: 2.368967]
149 [D loss: 0.991270, acc.: 65.62%] [G loss: 1.748382]
150 [D loss: 0.439574, acc.: 81.25%] [G loss: 2.044587]
151 [D loss: 0.233908, acc.: 89.06%] [G loss: 3.001447]
152 [D loss: 0.095637, acc.: 98.44%] [G loss: 3.110675]
153 [D loss: 0.100575, acc.: 98.44%] [G loss: 3.399302]
154 [D loss: 0.128999, acc.: 98.44%] [G loss: 3.435409]
155 [D loss: 0.118845, acc.: 98.44%] [G loss: 3.302231]
156 [D loss: 0.131589, acc.: 95.31%] [G loss: 3.201133]
157 [D loss: 0.159595, acc.: 92.19%] [G loss: 3.024340]
158 [D loss: 0.092949, acc.: 98.44%] [G loss: 2.916131]
159 [D loss: 0.105807, acc.: 98.44%] [G loss: 3.020361]
160 [D loss: 0.084535, acc.: 100.00%] [G loss: 3.009049]
161 [D loss: 0.140700, acc.: 93.75%] [G loss: 2.927458]
162 [D loss: 0.113794, acc.: 96.88%] [G loss: 2.918005]
163 [D loss: 0.121943, acc.: 98.44%] [G loss: 3

301 [D loss: 0.723615, acc.: 51.56%] [G loss: 0.862156]
302 [D loss: 0.723060, acc.: 45.31%] [G loss: 0.893823]
303 [D loss: 0.757806, acc.: 43.75%] [G loss: 0.837789]
304 [D loss: 0.748714, acc.: 46.88%] [G loss: 0.733382]
305 [D loss: 0.770583, acc.: 42.19%] [G loss: 0.772331]
306 [D loss: 0.698989, acc.: 48.44%] [G loss: 0.881004]
307 [D loss: 0.728675, acc.: 39.06%] [G loss: 0.881440]
308 [D loss: 0.786635, acc.: 34.38%] [G loss: 0.709624]
309 [D loss: 0.704561, acc.: 46.88%] [G loss: 0.761086]
310 [D loss: 0.689837, acc.: 51.56%] [G loss: 0.832635]
311 [D loss: 0.677678, acc.: 50.00%] [G loss: 0.843222]
312 [D loss: 0.694256, acc.: 48.44%] [G loss: 0.809055]
313 [D loss: 0.728835, acc.: 40.62%] [G loss: 0.744980]
314 [D loss: 0.674982, acc.: 50.00%] [G loss: 0.788844]
315 [D loss: 0.710323, acc.: 51.56%] [G loss: 0.777166]
316 [D loss: 0.769742, acc.: 35.94%] [G loss: 0.712157]
317 [D loss: 0.734891, acc.: 40.62%] [G loss: 0.713301]
318 [D loss: 0.690989, acc.: 53.12%] [G loss: 0.

...

29362 [D loss: 0.619052, acc.: 65.62%] [G loss: 0.895120]
29363 [D loss: 0.664536, acc.: 56.25%] [G loss: 0.981915]
29364 [D loss: 0.633644, acc.: 57.81%] [G loss: 0.987392]
29365 [D loss: 0.579626, acc.: 76.56%] [G loss: 0.923563]
29366 [D loss: 0.657444, acc.: 56.25%] [G loss: 0.928914]
29367 [D loss: 0.601321, acc.: 65.62%] [G loss: 0.953829]
29368 [D loss: 0.608714, acc.: 65.62%] [G loss: 1.086250]
29369 [D loss: 0.571726, acc.: 64.06%] [G loss: 1.099998]
29370 [D loss: 0.625947, acc.: 60.94%] [G loss: 0.989895]
29371 [D loss: 0.562115, acc.: 67.19%] [G loss: 1.147586]
29372 [D loss: 0.579215, acc.: 68.75%] [G loss: 1.059578]
29373 [D loss: 0.598513, acc.: 65.62%] [G loss: 1.112177]
29374 [D loss: 0.655079, acc.: 60.94%] [G loss: 1.048934]
29375 [D loss: 0.616004, acc.: 68.75%] [G loss: 1.014935]
29376 [D loss: 0.604597, acc.: 70.31%] [G loss: 1.025383]
29377 [D loss: 0.669173, acc.: 62.50%] [G loss: 0.998874]
29378 [D loss: 0.616957, acc.: 62.50%] [G loss: 1.110117]
29379 [D loss:

29516 [D loss: 0.571494, acc.: 71.88%] [G loss: 1.113017]
29517 [D loss: 0.514479, acc.: 78.12%] [G loss: 1.156032]
29518 [D loss: 0.703233, acc.: 59.38%] [G loss: 0.959668]
29519 [D loss: 0.558666, acc.: 71.88%] [G loss: 0.999840]
29520 [D loss: 0.593051, acc.: 68.75%] [G loss: 0.956828]
29521 [D loss: 0.650782, acc.: 62.50%] [G loss: 1.041381]
29522 [D loss: 0.581373, acc.: 67.19%] [G loss: 0.974049]
29523 [D loss: 0.563724, acc.: 62.50%] [G loss: 0.917147]
29524 [D loss: 0.555277, acc.: 73.44%] [G loss: 1.010897]
29525 [D loss: 0.688502, acc.: 57.81%] [G loss: 0.978489]
29526 [D loss: 0.673621, acc.: 59.38%] [G loss: 0.952894]
29527 [D loss: 0.658558, acc.: 64.06%] [G loss: 0.942058]
29528 [D loss: 0.643274, acc.: 59.38%] [G loss: 0.897601]
29529 [D loss: 0.638837, acc.: 62.50%] [G loss: 1.140079]
29530 [D loss: 0.650587, acc.: 62.50%] [G loss: 1.055149]
29531 [D loss: 0.732016, acc.: 53.12%] [G loss: 0.957416]
29532 [D loss: 0.634065, acc.: 62.50%] [G loss: 1.022996]
29533 [D loss:

29660 [D loss: 0.659734, acc.: 60.94%] [G loss: 1.158895]
29661 [D loss: 0.606452, acc.: 65.62%] [G loss: 1.140158]
29662 [D loss: 0.724085, acc.: 59.38%] [G loss: 1.162077]
29663 [D loss: 0.614982, acc.: 64.06%] [G loss: 1.077260]
29664 [D loss: 0.564031, acc.: 75.00%] [G loss: 1.037097]
29665 [D loss: 0.638941, acc.: 65.62%] [G loss: 1.127003]
29666 [D loss: 0.643573, acc.: 64.06%] [G loss: 1.120070]
29667 [D loss: 0.646022, acc.: 65.62%] [G loss: 0.933159]
29668 [D loss: 0.711301, acc.: 62.50%] [G loss: 1.070326]
29669 [D loss: 0.732784, acc.: 59.38%] [G loss: 1.026941]
29670 [D loss: 0.594014, acc.: 64.06%] [G loss: 1.040289]
29671 [D loss: 0.587564, acc.: 68.75%] [G loss: 0.991215]
29672 [D loss: 0.639519, acc.: 62.50%] [G loss: 0.988694]
29673 [D loss: 0.677718, acc.: 57.81%] [G loss: 0.923853]
29674 [D loss: 0.573207, acc.: 67.19%] [G loss: 0.996461]
29675 [D loss: 0.686523, acc.: 53.12%] [G loss: 1.098873]
29676 [D loss: 0.546850, acc.: 70.31%] [G loss: 1.061606]
29677 [D loss:

29816 [D loss: 0.652058, acc.: 64.06%] [G loss: 1.118562]
29817 [D loss: 0.612054, acc.: 68.75%] [G loss: 1.169786]
29818 [D loss: 0.712425, acc.: 59.38%] [G loss: 1.062920]
29819 [D loss: 0.647929, acc.: 60.94%] [G loss: 1.021872]
29820 [D loss: 0.669758, acc.: 57.81%] [G loss: 1.050930]
29821 [D loss: 0.729955, acc.: 56.25%] [G loss: 1.155472]
29822 [D loss: 0.709695, acc.: 64.06%] [G loss: 1.086043]
29823 [D loss: 0.564500, acc.: 73.44%] [G loss: 0.978903]
29824 [D loss: 0.560511, acc.: 76.56%] [G loss: 1.056583]
29825 [D loss: 0.685533, acc.: 57.81%] [G loss: 0.868095]
29826 [D loss: 0.591294, acc.: 64.06%] [G loss: 0.972007]
29827 [D loss: 0.657744, acc.: 56.25%] [G loss: 0.942461]
29828 [D loss: 0.619304, acc.: 68.75%] [G loss: 1.065633]
29829 [D loss: 0.573111, acc.: 67.19%] [G loss: 1.046316]
29830 [D loss: 0.629399, acc.: 65.62%] [G loss: 1.099146]
29831 [D loss: 0.532491, acc.: 76.56%] [G loss: 1.051182]
29832 [D loss: 0.619156, acc.: 64.06%] [G loss: 1.044489]
29833 [D loss:

29964 [D loss: 0.656355, acc.: 60.94%] [G loss: 1.093717]
29965 [D loss: 0.585778, acc.: 73.44%] [G loss: 0.922346]
29966 [D loss: 0.662265, acc.: 59.38%] [G loss: 0.949335]
29967 [D loss: 0.667240, acc.: 56.25%] [G loss: 1.102722]
29968 [D loss: 0.618885, acc.: 68.75%] [G loss: 1.093029]
29969 [D loss: 0.627377, acc.: 64.06%] [G loss: 0.980421]
29970 [D loss: 0.591552, acc.: 73.44%] [G loss: 1.045292]
29971 [D loss: 0.643667, acc.: 64.06%] [G loss: 1.230413]
29972 [D loss: 0.714441, acc.: 53.12%] [G loss: 1.137931]
29973 [D loss: 0.631500, acc.: 60.94%] [G loss: 1.052924]
29974 [D loss: 0.651317, acc.: 59.38%] [G loss: 1.127904]
29975 [D loss: 0.644699, acc.: 65.62%] [G loss: 0.986526]
29976 [D loss: 0.657725, acc.: 60.94%] [G loss: 0.961943]
29977 [D loss: 0.613926, acc.: 62.50%] [G loss: 1.031862]
29978 [D loss: 0.593340, acc.: 70.31%] [G loss: 0.895694]
29979 [D loss: 0.617071, acc.: 68.75%] [G loss: 1.112578]
29980 [D loss: 0.639930, acc.: 62.50%] [G loss: 1.054858]
29981 [D loss: